# Milestone 2
## Team 31

------

### Introduction

Automatic differentiation is a widely used tool for optimization, encompassing a set of efficient techniques used to computationally evaluate a function's derivative or gradient. At its core, automatic differentation exploits the idea that every function can be broken down into a sequence of elemenatry arithmetic operations and functions. Applying the chain rule to these operations in succession allows us to evaluate the derivatives we seek to high precision, without rampantly growing the number of arithmetic operations the computer needs to perform. As functions grow in size, complexity, and dimension, the need for an efficient algorithm to accurately derive the gradient becomes clear. The goal of our project is to develop a software library for automatic differentiation with both a forward mode and a reverse mode implementation to provide such efficient and accurate derivative calculations.

The applications of automatic differentiation are vast and ever-expanding. Ranging from finding the solution to ordinary differential equations (ODEs) and optimizing objective functions, to finding the solution of linear systems as an alternative to iterative methods, automatic differentiation is a highly useful technique. Many popular machine learning libraries such as PyTorch make use of such automatic differentiation. Prominent example applications span essentially any gradient-based optimization algorithm, including Newton’s method, training neural networks, sampling high-dimensional Bayesian model posteriors, and Hamiltonian Monte Carlo samplers $^{(1)}$. Speeding up computation and eliminating round-off error is crucial in applying modern gradient-based optimization algorithms across science and engineering. Automatic differentiation, as opposed to alternatives such as symbolic and numerical differentiation, holds many advantages in terms of both its efficiency and accuracy. As models – especially in neural network-based machine learning – become more complex and high-dimensional data becomes more readily available, the ability efficiently to compute derivatives is only becoming more important.

$^{(1)}$ *A Review of Automatic Differentiation and its Efficient Implementation* (Margossian)

------

### Background

#### Forward Mode
Automatic differentiation is a set of techniques to computationally determine a function's gradient, centered around an iterative evaluation of the chain rule.  This piecewise evaluation of the chain rule to find a derivative stems from the idea of breaking down complex functions into a sequence of elementary operations (addition, multiplication, etc.) and elementary functions (sin, exp, etc.), from the inside of the function outwards.  This sequence of elementary operations and functions that string together to create the function definition gives rise to the forward primal trace of the function.  Using our function inputs and evaluating through this sequence, we can see that at each stage in the sequence we will get an intermediate result $v_j$ that is dependent on our inputs.

The breakdown of a complex function into this sequence of elementary operations and functions (from the inside out) is most easily visualized in what we call the computational forward graph.  Our inputs $x_k$ to the original complex function represents nodes $v_{k-m}$ for $k$ = 1,2,...,m.  As we pass through the sequence, each intermediate results $v_j$ is alloted a node while the directed edges connecting nodes represent the elementary operation/function that leads to that intermediate result (a new node on the graph).

To compute the given derivative of the complex function we need to define a differential operator $D_p(.)$ representing the directional derivative in the direction of $p$ (note: $p$ is the seed vector that the user chooses if they desire a certain derivative of interest). Then, when we evaluate the forward primal trace of the function (finding all the intermediate values $v_j$), we also simultaneously evaluate (and carry forward) $D_p v_j$, the derivative of $v_j$ in direction of $p$.  Thus, for each step in the sequence we get both an intermediate value $v_j$ based on previous $v_j$'s  from the forward primal trace but also a value $D_p v_j$ based on the previous $D_p v_j$'s.  The sequence of formulas that define $D_p v_j$ at every step for a given value of p is called the forward tangent trace. Every time we wish to compute the derivative in a new direction we must perform a new pass through the forward tangent trace, plugging in a different $p$ direction vector to our $D_p v_j$'s at each step to find their values.

An important aside in the calculation of our gradients is the application of the chain rule touched on above. The chain rule allows the representation of a functions derivative in terms of the composition of multiple differentiable functions. Beginning with a simple example, we can consider a function in the form $f(g(t))$.  As a result of the chain rule we write its derviative as $\frac{df}{dt} = \frac{df}{dg} \cdot \frac{dg}{dt}$. We can expand this concept to higher dimensions to further generalize our chain rule. With a multivariate function, it is important to consider the gradient, which represents the summation of the partial derivatives of f to its inputs. We can thus write our generalized chain rule as follows: $\nabla_x f = \sum_{i=1}^n \frac{\partial f}{\partial y_i} \nabla y_i (x)$.

One structure that is critical in the implementation of forward mode AD is dual numbers.  A dual number $z$ takes the form $z = a + b\epsilon$ where $a$ is the real part, $b$ is the dual part, and $\epsilon$ is a nilpotent number such that $\epsilon^2 = 0$ (note: $\epsilon$ does not equal 0). They have three very useful properties:
1. Adding dual numbers together resembles the linearity of addition and results in adding the functions in the real part and adding the derivatives in the dual part
2. Multiplication results in multiplication of the functions in the real part and the product rule for the derivatives in the dual part
3. Chain rule still holds for dual numbers

Because of these properties, dual numbers can be used to efficiently encode the primal and tangent trace of intermediate results $v_i$ and $v_j$ as we pass forward through the sequence (computational graph).  Letting $z_j = v_j + D_p v_j \epsilon$ for intermediate results, we are able to conveniently store the primal trace and the tangent trace in the real and dual part (respectively) of a dual number ($z_j$) allowing us to carry them forward through the sequence as a pair.


#### REVERSE MODE
While the forward mode implementation described above holds many advantages for the evaluation of a function and its gradient, it has shortcomings when dealing with functions of high dimensions. In particular, for functions $f: \mathbb{R}^m:\mathbb{R}$ for large $m$, forward mode requires $m$ passes to determine the gradient of $f$. This is a common scenario for neural networks, and is where the alternative technique of reverse mode excels.

Reverse mode, as opposed to forward mode, requires only two passes to determine the gradient of a function. The first pass, named the forward pass, computes the primal trace $(v_j)$ as well as the partial derivatives with respect to the parent nodes. However, unlike in forward mode, the chain rule must not be explicitly applied. Instead, we then perform the reverse pass, where the chain rule will be reconstructed by accumulating values as we traverse backwards across our computational graph. The result is a calculation of the gradient, with a computational cost not dependent on $m$ as forward mode is. However, reverse mode does have limitations in the sense that we must store the computational graph of our function which may be memory intensive. For a forward mode implementation, we iterate over the graph sequentially and do not need to return to prior nodes, whereas for the reverse pass in our reverse mode implementation these previous nodes must be stored as they are later revisited. Hence, reverse mode requires more memory allocation in terms of storing the computational graph of our function, but offers a substantial advantage in terms of its computational cost for functions where high dimension inputs. We will provide reverse mode as an extension of our automatic differentiation library.


-----

### How to Use Autodiff

#### Autodiff Installation

Our package is available for installation via TestPyPI. We decided to use TestPyPI as our project is still in experimentation stages and felt this would be the better suited option. Below are the detailed steps for creating a virtual environment in which to install and use our package effectively.

1. Navigate to desired directory and create virtual environment
```python
python -m venv test_env
```
2. Activate the environment 
```python
source test_env/bin/activate
```
3. Navigate inside test_env and install dependencies
```python
cd test_env
python -m pip install numpy
```
4. Install our package
```python
python -m pip install -i https://test.pypi.org/simple/ autodiff-team31==0.0.1
```
5. Write your code and import our package!
```python
>>> import autodiff as ad
>>> ad.sin(1)
0.8414709848078965
# More examples below
```
6. Deactivate the environment 
```python
deactivate
``` 


#### Autodiff in Practice

The core of our autodiff library is the Func class. Through its use, users can use forward mode AD to find the jacobian of a function at a given point or to evaluate both the function value and the derivative in a given direction at a given point.  Steps for this, with example code, are outline below:

1. Import autodiff along with its Func module
```python
>>> import autodiff as ad
>>> from autodiff.func import Func
```
2. Define the function they wish to differentiate using traditional Python function defintions or lambda functions. Note: if the function definition involves sines, cosines, exponents, etc., users should use the autodiff functions (ad.sin, ad.cos, ad.exp, etc.) in their function defintion.
```python
>>> def g(x, y):
        return y + x**2, x - 3*y
```
3. Instantiate a Func object.  Note: Func takes 3 parameters:
    * function : function object
        * The user-defined mathematical function that users wish to differentiate.
    * num_inputs : int
        * The number of inputs to the function users wish to differentiate.
    * num_outputs : int
        * The number of outputs to the function users wish to differentiate.
```python
>>> gofx = Func(g,2,2)
```
4. 
    * To find the jacobian users can call the 'jacobian' method of their Func instance.  Note: jacobian takes 1 parameter:
        * point : int, float, list, np.ndarray
            * The input point at wish users wish to evaluate the function's jacobian. Can be int/float if & only if input dimension = 1.
    * This method returns the jacobian of the function at the inputted point. When the jacobian is a scalar, it returns a float.  Otherwise, it returns a num_outputs by num_inputs numpy array. <br><br>   
    * To evaluate the function value and the derivative value users can call the 'eval' method of their Func instance. Note: eval takes 2 parameters:
        * point : int, float, list, np.ndarray
            * The input point at wish users wish to evaluate the function and the function's derivative. Can be int/float if & only if input dimension = 1.
        * seed_vector : list, np.ndarray
            * The direction in wish users wish to evaluate the function's derivative
    * This method returns a tuple.  The first element is an np.ndarray with num_outputs' length representing the function evaluated at the given point.  The second element is an np.ndarray with num_outputs' length representing the value of the derivative at the given point in the given direction.           
```python
>>> gofx.jacobian([4,2])
array([[ 8.,  1.],
       [ 1., -3.]])
>>> gofx.eval([4,2], [1,0])
(array([18., -2.]), array([8., 1.]))
```            


#### Demo - Newton's Method

This demo will show how the Newton's method algorithm can utilize autodiff's differentiation capabilities to find roots for the given function:
  
## $$ f(x) = x - e^{-2(\sin(4x)) ^2} $$

#### $f(x)$ has roots at $x$ $\approx$ 0.2474, $x$ $\approx$ 0.6692, and $x$ $\approx$ 0.8560

In [4]:
import autodiff as ad
from autodiff.func import Func

def newton(f, J, x_k, tol=1.0e-8, max_it=100):
    root = None
    for k in range(max_it):
        dx_k = -f(x_k) / J(x_k)
        if abs(dx_k) < tol:
            root = x_k + dx_k
            print(f"Found root {root:e} at iteration {k+1}\n")
            break
        #print(f"Iteration {k+1}: Delta x = {dx_k:e}")
        x_k += dx_k

        
f = lambda x: x - ad.exp(-2.0 * ad.sin(4.0 * x) * ad.sin(4.0 * x))

#utilization of autodiff
fofx = Func(function = f, num_inputs=1, num_outputs = 1)
J = fofx.jacobian

print('With Initial Guess: x = 0.1:')
newton(f, J, 0.1)
print('With Initial Guess: x = 0.6:')
newton(f, J, 0.6)
print('With Initial Guess: x = 0.9:')
newton(f, J, 0.9)

With Initial Guess: x = 0.1:
Found root 2.473652e-01 at iteration 5

With Initial Guess: x = 0.6:
Found root 6.692328e-01 at iteration 5

With Initial Guess: x = 0.9:
Found root 8.560317e-01 at iteration 4



-------

### Software Organization

This section provides a high-level overview of how our software is organized.  
  
Our directory structure is as follows – autodiff is the name of our package:
```
team31 
├── LICENSE    
├── pyproject.toml      
├── README.md
├── setup.cfg
├── docs
│    ├── milestone1.ipynb
│    ├── milestone2.ipynb
│    └── milestone2_progress.md
├── src
│    └── autodiff
│            ├── __init__.py 
│            ├── dualnumber.py 
│            ├── func.py
│            └── operators.py 
└── test
     ├── run_tests.sh
     ├── check_coverage.sh
     ├── test_dualnumber.py
     ├── test_func.py 
     ├── test_operators.py 
     ├── parser.py
     └── coverage.xml


```



Our AD library is built on 3 central modules:

- DualNumber
    - Within the dualnumber module we define the DualNumber class which, although never accessed directly by the user, forms the foundation of our forward mode implementation. We specify in the class constructor the attributes that uniquely identify dual numbers, the real (self.real) and dual (self.dual) parts.  Then we overload all the operators for basic arithmetic (+, -, \*, /, etc.) and their reverse options to ensure that DualNumber instances can be used in user defined functions.  Throughout the class we provide checks for the type of inputted or interacting objects as we intend to support int and float operations paired with dual numbers but raise errors otherwise.  Specific implementation details are further described below.
- Func
    - Within the func module we define the Func class, the heart of our forward mode implementation. Func class objects take as input, and store as attributes, the function a user wishes to differentiate, the number of inputs, and the number of outputs.  Func objects have two key methods: jacobian() and eval().  Calling Func_instance.jacobian(point), with a point as an inputted returns the jacobian matrix (or scalar in the case of a scalar function) of the function at the inputted point.  Calling Func_instance.eval(point,seed_vector) returns a tuple where the first element is an array representing the function's value at the inputted point and the second element array representing the value of the derivative at the inputted point in the direction of the inputted seed_vector.  Throughout the class we implement checks to raise errors if we encounter dimensions mismatches or invalid inputs. Specific implementation details are further described below.
- Operators
    - Within the operators module we define several functions (sin, exp, log, etc.) to specify how basic unary operations should be carried out during function evaluation in forward mode AD.  Notably, we designate how these operators should act on DualNumber instances, on scalars (ints/floats), and between DualNumber instances and scalars (ints/floats).  These defined functions allow users to define the functions they wish to differentiate with operators compatible with our AD library in an intuitive way.  There are also checks within this class to determine the type of input such that each function returns the desired result or, in the case of an invalid input, an error. Specific implementation details are further described below.


Test Suite:

Our tests reside in a separate test directory that reflects our src directory. We have created a script, run_tests.sh, to run all tests within our directory with pytest. We have extensive unit tests implemented for the DualNumber class, operators, and the Func class. We also have developed some, and will develop more integration tests that use our func module and ensure the behavior is correct when deriving a jacobian for complex functions. Additionally, we have created a coverage check, runnable via the script check_coverage.sh, that will generate a coverage report that is passed into a coverage.xml file. From here, we have also created a parser python file that extracts the coverage percentage from this coverage.xml file and determines whether it meets our threshold of 90%. This is run by our coverage workflow, which calls the check_coverage.sh script followed by parser.py to determine if we have met our coverage criteria. Currently we have achieved a coverage of 98%. Similarly we have a workflow that ensures all tests passed by running the run_tests.sh script.


Package Installation:

Our package is available for installation via TestPyPI with PEP517/518. As such, we have included the required pyproject.toml file with our package metadata, as well as a setup.cfg file as we use setuptools for our backend. This enables easy building and distribution of our package.  Please refer to the **How to Use Autodiff - Autodiff Installation** section for a step by step guide, with example code, of how to install autodiff on your machine.

------

### Implementation Details

The current (forward mode) implementation of our AD library relies on the integration and interaction between our 3 core modules: dualnumber, func, and operators.  Broadly, dualnumber provides the DualNumber class, func defines the Func class, and operators handles elementary function defintions.

- Core classes and Important Attributes
<br><br>
    - The current core classes in autodiff are the DualNumber class and the Func class, defined in their namesake modules.  The attributes and methods of these classes allows them to work together in performing forward mode AD.<br>

   **DualNumber()**
<br><br>
        - The main purpose behind DualNumber is to create a class of objects to encode the current state of a dual number (a+b$\epsilon$, where $\epsilon^2$ == 0)  such that it can be used in function operations, as is needed in automatic differentiation.  The class constructor takes two arguments – real and dual – representing the real and dual parts of the specified dual number.  These values are stored as attributes of the instance, self.real and self.dual, such that they can be accessed later by class methods ot be updated or used in the instantiation of a new DualNumber object.  Within DualNumber, we implement a series of dunder methods to overload basic operators, comparative operators, and functions  (+, -, \*, /, pow, etc.), along with their reverse options, to accurately reflect how arithmetic between dual numbers themselves and between dual numbers and scalars (ints/floats) is performed.  When carrying out operations between two DualNumbers, the result's real part is created by performing the specified operation between both of the original's real parts. The dual part, however, is created by performing the first order differentation of said specified operation.  Because this is not how Python naturally implements these basic operators, it was essential for us to overload in order to ensure compatibility with dual number arithmetic.  Throughout the class there are a series of checks to ensure that any inputs to the methods are of valid type (ints, floats, or dual numbers).<br><br>
        
   **Func()**
   <br><br>
        - Func class instances serve as the clients main point of interaction with our AD library.  The class constructor takes 3 inputs: function, num_inputs, and num_outputs. Function is the defined function the users wishes to differentiate and will be stored as an attribute in self.func (since functions are first class objects in Python). Num_inputs and Num_outputs represent the respective input and output dimension of inputted function and will be stored in the attriutes self.num_inputs and self.num_outputs.  These attributes allow Func to internally initialize NumPy arrays of the correct size for storing jacobian or derivative evaluations and make it possible to refer back to the user defined function when calculating values and derivatives.  Within the function class we also define two methods: jacobian and eval.
       <br> <br>
        Jacobian() takes a point p, the point at which users want to evaluate the jacobian of the function as an input (p has dimension num_inputs).  Internally it creates an identity matrix of dimension num_inputs by num_inputs where each column represents a seed vector.  At the same time it initializes a num_outputs by num_inputs matrix of zeros which will store the jacobian values. A for loop then iterates through the columns of the identity matrix, setting that iteration's column to be the seed_vector of choice. In each loop, jacobian() initializes num_inputs dual numbers from the DualNumber class where dual number i is: $z_i$ = DualNumber(p[i],seed_vector[i]). It then calls self.func($z_1$...$z_{num\_inputs}$) which returns a dual number (or sequence of dual numbers for multiple output functions), where the real part of the dual number(s) is the value of the function at the input point p and its dual part is the derivative of the function at point p in direction seed_vector. We will then store this derivative value (or vector if there are multiple outputs) in the corresponding column of our previously initialized jacobian matrix. Once we have looped through all the seed_vectors (and thus all the inputs), we will have found the values of the derivative of each output of the function with respect to each input of the function at the point p and stored them in the jacobian matrix.  We also check if the jacobian is a scalar (a 1x1 matrix) in which case we return just the scalar value. Otherwise, the num_outputs by num_inputs jacobian matrix is returned.<br> <br>
        Eval() takes two inputs, a point p (the point where users want to evaluate the function and derivative) and a direction (seed_vector), the direction in which users want to evaluate the directional derivative (eg. seed_vector = [1,0,0,0]).  Like the jacobian() method, internally the eval() method will also define num_inputs dual numbers $z_i$, where $z_i$ = DualNumber(p[i],seed_vector[i]).  Eval() then calls self.func($z_1$...$z_{num\_inputs}$) to generate a dual number (or sequence of dual numbers for multiple output functions), where the real part of the dual number(s) is the value of the function at the input point p and its dual part is the derivative of the function at point p in direction seed_vector. The real part(s) of the(se) dual number(s) is stored in an array called val and the dual part is stored in an array called deriv.  Finally, the method returns the val array and deriv array in a tuple where the first element (val) represents the function's value evaluated at the inputted point and the second element (deriv) represents the value of the derivative at the inputted point in the inputted direction.  Throughout the entire class we implement checks to raise errors if we encounter dimensions mismatches or invalid inputs (eg. inputted points must be int, float, list, or NumPy Arrays; strings will raise an error). 
<br><br>

    __Operators__
 <br><br>
        - While our operators module is not a class, it is an important aspect of our implementation as we define functions such as sin, cos, tan, log, and exp. Here we make use our numpy dependency and define these elementary functions for ints, floats, and dual numbers. For the dual number implementation, we utilize the chain rule to properly evaluate the dual component and return a new dual number object. As a result of our implementation of these operators, we allow users to make functions using syntax such as ad.sin(x), which is much more intuitive and reflective of a numpy implementation. Additionally, we include checks to ensure the input is of the proper type.
    
    <br><br>
- Core data structures
    - The key data structures used in autodiff are lists and NumPy arrays.  Most notably, for functions with multiple inputs and outputs, the parameters to the methods in our Func class will have to be lists or np.ndarrays.  For higher dimensional functions, our Func methods also return the elements of the output – a jacobian matrix for the jacobian() method and the function value and directional derivative for the eval() method – as np.ndarrays.  This is because many functions in the NumPy library are already highly optimized for np.ndarray objects.  If users wanted to transform the output of their AD call (such as find the inverse of the jacobian), they could easily use NumPy functions on the returned objects without worries of suboptimal performance.  Internally, a list is used to store the DualNumber object inputs that are generated from the user inputted point and seed_vector (see Func implementation for details).  This is convenient as when we call self.func() on the list of DualNumbers objects, we can simply use the * to unpack the list items to serve as inputs to the function.<br><br>
    - As we plan for our reverse mode implementation and computational graph display, we'll have to make use of another data structure: a directed acyclic graph (DAG), implemented using the graphviz package. This will store the forward trace of the elementary functions. We will also be able to access the primals at each computational step. Additionally, given our support for directional differentiation at a given point, our DAG will also be able to store our seed vector and the point in question. A topological sort will also be enacted in order to provide a streamlined view of our operations; in other words, we would like to make it clear which operations are prioritized when "working from the inside out."
<br><br>
- External dependencies
<br><br>
    - For our AD library, we will rely on the NumPy module as our sole external dependency. NumPy is a package which is extremely resourceful for supporting arrays, matrices, and other mathematical operators which are omitted from the native pythonic library. It was necessary for importing sin, cosine, and tangent functions which are used in writing the DualNumber operators in our operator.py file. Additionally, we introduce the numpy natural log function which is essential for overloading our "pow" and "rpow" methods in the DualNumber class; it is also used to support the logarithm and exponent functions in operators.py. Lastly, as stated before, numpy provides us with the nd.array type which can be used to create matrices of any shape. We use the np.zeros and np.identity methods for implementing our Jacobian and derivative evaluation functions in func.py. It is also important to highlight that our functions support numpy arrays as input values given our dependency on the module. We decided to use numpy over the pythononic math module since numpy allows for the handling of array inputs; the math function methods only support scalar inputs, which would make the scope of our package much more narrow.
<br><br>  
  
   
   
 - Future Implementation Plans
 <br><br>
    - We have yet to implement our revDualNumber class, an modification of the DualNumber class to make it compatible with reverse mode AD.  We anticipate instantiating this class with the same inputs as the DualNumbers, but with additional parent and operation inputs as well.  These supplementary attributes are needed to create a computational graph that will allow us to perform the reverse pass. For example, we could keep an create an attribute self.op to keep track of the operation type performed on with that dual number, a self.parents attribute to keep track of the parents of that dual number, and a self.name attribute that defines the intermediate result $v_j$ (which becomes the node on the graph) represented by the dual number.  This would force us to redefine some of our overloaded operator methods as well such that the new methods would instantiate the dual numbers representing intermediate nodes correctly.  We also have thought to add a backwards function to all of our overloaded methods that represents the derivative of the elementary functions defined by each overloaded method. Keeping track of the names of nodes associated with dual numbers along with the operations and parents of nodes, should allow us to perform the reverse pass necessary in reverse mode AD.  Within the RevDualNumber class we can then create a graph_trace method that uses a topological sorting algorithm to follow each parent DualNumber backwards recursvely as far as possible while still maintaining the correct order for any intermediate dependencies. This method should return the sorted order of the graph nodes along with the nodes and edges we need to visualize the graph, something we can implement by using Python's graphviz library. To actually compute the reverse path, we define a final backwards method that uses the sorted order of graph nodes and the earlier backwards functions. By calling this method we would compute the reverse pass and return the gradients. While these implementation details may be altered throughout the production process, it will not affect the user as users do not interact directly with the revDualNumber class (just as they do not interact directly with the DualNumber class).
<br><br>
    - Users instead interact directly with Func class objects.  Thus, we plan to extend the jacobian and eval methods in the Func class with two new boolean inputs – reverse (set to False by default) and return_graph (set to False by default).  If these are false, derivatives will be calculated using forward mode AD, with DualNumber objects under the hood.  If reverse is set to True and return_graph is set to false, derivatives will be calculated using reverse mode AD, with revDualNumber objects under the hood.  If they are both set to true, derivatives will be calculated using reverse mode AD, with revDualNumber objects under the hood, and a graph object representing computational graph of the function will also be returned.
<br><br>
    - This brings us to our final needed implementation, a draw_graph function.  This function will utilize the Digraph function from the graphviz package. It should take the returned graph object with information about the nodes and edges from running Func_instance.jacobian() or Func_instance.eval() with reverse = True and return_graph = True as input; then it should display the computational graph of the function.

-----

### Future Features

Our future features center around the following three goals:

__1. Reverse Mode Implementation__

We are currently prioritizing the implementation of reverse mode automatic differentiation. Reverse mode has great efficiency benefits when dealing with functions from $\mathbb{R}^m \to \mathbb{R}^n$ for $m>>n$, as its efficiency is not dependent on the number of independent variables of a function. Hence, for many machine learning or optimization applications our package can be used for, this will be very important. We are debating the creation of a specific dual number class specifically for the reverse mode implementation, versus adding an optional parameter to our existing dual number class for this. While integrating this into our existing dual number class would allow for some code reuse, it may make our dual number module less clear and easily maintainable. Hence, we are currently leaning towards creating a separate "rev_dualnumber" module. When instantiating a function object for reverse mode, we intend users to pass in an optional parameter specifying they intend to use reverse mode. This will allow us to reuse the func module for reverse mode. Implementing reverse mode will require a forward and reverse pass, and we must also store the state of our intermediate variables.

__2. Graph Visualization__

We are also intending on implementing graph visualization support. We plan on using the graphviz python package, which is also used by libraries such as Keras for neural network visualization. With this graphviz dependency, we will be able to use the Digraph class to generate the directed graph corresponding to our computational graph. We intend on creating a graph module to support the graphing of a computational graph for a particular function.

__3. Graph Optimization__ 

We also seek to implement a topological sorting algorithm for function evaluation and the reverse mode reverse pass. This will require the construction of a directed acylic graph structure from a given function. Rather than naively evaluating a function object as we currently do, creating a graph structure that we may apply a topological sorting algorithm to will result in substantial efficiency improvements.